In [1]:
import psycopg2

# Revision de la tabla de tags y de posts

In [2]:
connection = psycopg2.connect(user="postgres", password="postgres", host="127.0.0.1", port="63333", database="stack_overflow")

In [3]:
with connection:
    with connection.cursor() as cursor:
        cursor.execute("select tags from posts_19 where tags is not null limit 6")
        posts_tags_sample = cursor.fetchall()
posts_tags_sample

[('<c#><floating-point><type-conversion><double><decimal>',),
 ('<html><css><internet-explorer-7>',),
 ('<c#><.net><datetime>',),
 ('<c#><datetime><time><datediff><relative-time-span>',),
 ('<html><browser><timezone><user-agent><timezone-offset>',),
 ('<.net><math>',)]

In [4]:
with connection:
    with connection.cursor() as cursor:
        cursor.execute("select tag_name, count from tags order by count desc limit 6")
        top_tags_sample = cursor.fetchall()
top_tags_sample

[('javascript', 2453736),
 ('python', 2071403),
 ('java', 1878099),
 ('c#', 1571218),
 ('php', 1451338),
 ('android', 1393144)]

In [5]:
with connection:
    with connection.cursor() as cursor:
        cursor.execute("select tag_name from tags where tag_name like '%<%' or tag_name like '%>%'")
        tags_with_angle_brackets = cursor.fetchall()
tags_with_angle_brackets

[]

# Crear la tabla de posts_tags

In [6]:
with connection:
    with connection.cursor() as cursor:
        cursor.execute("""
        with posts_tags_exp as (
            select id, regexp_split_to_table(trim('<>' from tags), '><') as i_tag
            from posts_19 where tags is not null limit 10
        )
        select pt.id, pt.i_tag, t.id
        from posts_tags_exp pt
        left join tags t on pt.i_tag=t.tag_name
        """)
        posts2tags = cursor.fetchall()
posts2tags

[(9, '.net', 1),
 (6, 'html', 2),
 (6, 'css', 4),
 (9, 'c#', 9),
 (4, 'c#', 9),
 (4, 'floating-point', 305),
 (4, 'double', 451),
 (4, 'decimal', 6250),
 (4, 'type-conversion', 9780),
 (6, 'internet-explorer-7', 36930)]

In [19]:
with connection:
    with connection.cursor() as cursor:
        cursor.execute("""
        DROP TABLE IF EXISTS posts_19_tags;
        CREATE TEMP TABLE posts_19_tags AS
        with posts_tags_exp as (
            select id, regexp_split_to_table(trim('<>' from tags), '><') as i_tag
            from posts_19 where tags is not null
        )
        select pt.id as post_id, t.id as tag_id
        from posts_tags_exp pt
        left join tags t on pt.i_tag=t.tag_name
        """)

### Prueba de la relacion

In [23]:
with connection:
    with connection.cursor() as cursor:
        cursor.execute("""select t.tag_name, p.id, u.country
        from posts_19_tags pt
        join posts_19 p on pt.post_id=p.id 
        join tags t on pt.tag_id=t.id 
        join users u on p.owner_user_id=u.id 
        limit 20""")
        post_tags_country = cursor.fetchall()
post_tags_country

[('culture', 90061, None),
 ('globalization', 90061, None),
 ('.net', 90061, None),
 ('loops', 93511, 'Ireland'),
 ('xslt', 93511, 'Ireland'),
 ('xml', 93511, 'Ireland'),
 ('exists', 99132, 'Canada'),
 ('directory', 99132, 'Canada'),
 ('makefile', 99132, 'Canada'),
 ('html', 101574, 'United States'),
 ('.net', 112625, 'Australia'),
 ('auto-indent', 118728, None),
 ('indentation', 118728, None),
 ('notepad++', 118728, None),
 ('add-in', 126526, 'United Kingdom'),
 ('ms-office', 126526, 'United Kingdom'),
 ('vb.net', 126526, 'United Kingdom'),
 ('duplicate-removal', 150760, 'United States'),
 ('business-intelligence', 150760, 'United States'),
 ('duplicates', 150760, 'United States')]